In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from datetime import datetime
import os
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import numpy as np

In [7]:
postgres_pwd=os.environ['LHL_DATABASE']
con = psycopg2.connect(database='mid_term_project', 
                       user='lhl_student',
                       password = postgres_pwd,
                       host='mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com',
                       port='5432')

query = con.cursor()

In [6]:
query.close()

In [8]:
query.execute("""
select *
from flights_test
limit 500 """)

response = query.fetchall()
cols1 = [column[0] for column in query.description]

In [10]:
flights_test500 = pd.DataFrame(response,columns= cols1)
flights_test500.to_csv('flights_test500.csv',index = False)

In [11]:
with open('flights_test500.csv')as g:
    flights_19 = pd.read_csv(g)
headers = flights_19.columns.to_list()

In [15]:
with open('flights_test.csv')as f:
    flights = pd.read_csv(f,names=headers,header= None)

In [67]:
flights.head()
flights.to_csv('flights_headertest.csv',index = False)

In [106]:
columns_test = ['fl_date','mkt_carrier','mkt_carrier_fl_num','tail_num','origin_airport_id',
                'origin_city_name','dest_airport_id','dest_city_name','crs_dep_time','crs_arr_time'
                ,'crs_elapsed_time','distance','origin','dest']
test = flights[columns_test]

In [100]:
# conver= flights[['fl_date','mkt_carrier','mkt_carrier_fl_num',
#               'origin_airport_id','dest_airport_id','crs_dep_time','crs_arr_time','origin','dest']]

# conver['fl_date']= pd.to_datetime(conver['fl_date'])
# conver['month'] = conver.fl_date.dt.month
# conver['year'] = conver.fl_date.dt.year
# conver['day'] = conver.fl_date.dt.day
# conver.to_csv('merge_flight_test.csv',index = False)

In [88]:
test.groupby(['fl_date','mkt_carrier','mkt_carrier_fl_num',
              'origin_airport_id','dest_airport_id','crs_dep_time','crs_arr_time']).count().shape

(659057, 9)

In [107]:
test=test.dropna().reset_index(drop = True)
test['origin_state'] = test.origin_city_name.str.split(',').str[1]
test['origin_state'] = test['origin_state'].str.strip(' ')

In [22]:
def label_states(x):
    if x in ['ME', 'NH', 'VT', 'MA', 'RI', 'CT', 'NY', 'PA', 'ND','NJ']:
        'North-East'
        return 1
    elif x in ['ND', 'SD', 'NE', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH', 'KS']:
#         Nebraska NB or NE'Midwest'
        return 2
    elif x in ['DE','DC','VI','MD', 'WV', 'VA','PR','NC', 'KY', 'SC', 'TN', 'GA', 'FL', 'AL', 'AR', 'OK', 'TX', 'LA','MS']:
#         South'
        return 3
    elif x in ['WA','OR', 'CA', 'AK', 'NV', 'ID', 'MT', 'WY', 'UT', 'AZ', 'NM', 'CO','HI','TT']:
#         west
        return 4
    return 1000

In [108]:
test['origin_state_region']= test['origin_state'].apply(lambda x:label_states(x))

In [24]:
test['origin_state_region'].value_counts()

3    262102
4    194321
2    115696
1     86938
Name: origin_state_region, dtype: int64

In [25]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
labelenmkt = LabelEncoder()
labelentail = LabelEncoder()

In [110]:
test['mkt_carrier_Cat'] = labelenmkt.fit_transform(test['mkt_carrier'])
test['tail_num_Cat'] = labelentail.fit_transform(test['tail_num'])

In [116]:
a = test._get_numeric_data().columns.to_list()
a.extend(['fl_date','origin','dest','mkt_carrier'])

In [118]:
df = test[a]
df = df.drop(columns='distance')
df.head()

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,origin_state_region,mkt_carrier_Cat,tail_num_Cat,fl_date,origin,dest,mkt_carrier
0,5888,13891,14771,1810,1945,95,4,9,5513,2020-01-01 00:00:00,ONT,SFO,WN
1,6276,13891,14771,1150,1320,90,4,9,2243,2020-01-01 00:00:00,ONT,SFO,WN
2,4598,13891,14831,2020,2130,70,4,9,4013,2020-01-01 00:00:00,ONT,SJC,WN
3,4761,13891,14831,1340,1455,75,4,9,2602,2020-01-01 00:00:00,ONT,SJC,WN
4,5162,13891,14831,915,1035,80,4,9,5636,2020-01-01 00:00:00,ONT,SJC,WN


In [119]:
df['fl_date'] = pd.to_datetime(df.fl_date)
# 0 is for monday and 6 is sunday
df['month'] = df.fl_date.dt.month
df['dayof_theweek'] = df.fl_date.dt.dayofweek
df['year'] = df.fl_date.dt.year
df['day'] = df.fl_date.dt.day
df["WEEKEND"] = ((df.fl_date.dt.dayofweek) // 5 == 1).astype(float)

In [120]:
step1 = df.groupby(by=['origin_airport_id','mkt_carrier_Cat'],as_index= False)\
                                .agg({'dest_airport_id':'count'})\
                                .rename(columns={'dest_airport_id':'Nos_flight/carrier in airport'})
step2 = df.groupby(by=['origin_airport_id'],as_index= False)\
                                        .agg({'dest_airport_id':'count'})\
                                        .rename(columns={'dest_airport_id':'Nos flight per airport'})

In [121]:
new_df = pd.merge(df,step1,  how='inner', left_on=['origin_airport_id','mkt_carrier_Cat'],
                  right_on = ['origin_airport_id','mkt_carrier_Cat'])
new_df = pd.merge(new_df,step2,  how='inner', on='origin_airport_id')

In [33]:
def periodof_day(x):
    if  x >= 500 and x < 1200:
        return 1
    elif x >= 1200 and x < 1700:
        return 2
    elif x >= 1700 and x < 2100: 
        return 3
    return 4
# small,medium,large,Xlarge
def airport_size(x):
    if   x < 116:
        return 1
    elif x >= 116 and x < 407:
        return 2
    elif x >= 407 and x < 1518: 
        return 3
    return 4

def airline_size(x):
    if   x < 82:
        return 1
    elif x >= 82 and x < 198:
        return 2
    elif x >= 198 and x < 532: 
        return 3
    return 4

In [122]:
# for binning
new_df['Nos flight per airport']= new_df['Nos flight per airport'].apply(lambda x:airport_size(x))
new_df['Nos_flight/carrier in airport']= new_df['Nos_flight/carrier in airport'].apply(lambda x:airline_size(x))
new_df['periodof_day']= new_df.crs_dep_time.apply(lambda x:periodof_day(x))
print(new_df.shape)
new_df.head()

(659057, 21)


,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,origin_state_region,mkt_carrier_Cat,tail_num_Cat,fl_date,...,dest,mkt_carrier,month,dayof_theweek,year,day,WEEKEND,Nos_flight/carrier in airport,Nos flight per airport,periodof_day
0,5888,13891,14771,1810,1945,95,4,9,5513,2020-01-01,...,SFO,WN,1,2,2020,1,0.0,4,4,3
1,6276,13891,14771,1150,1320,90,4,9,2243,2020-01-01,...,SFO,WN,1,2,2020,1,0.0,4,4,1
2,4598,13891,14831,2020,2130,70,4,9,4013,2020-01-01,...,SJC,WN,1,2,2020,1,0.0,4,4,3
3,4761,13891,14831,1340,1455,75,4,9,2602,2020-01-01,...,SJC,WN,1,2,2020,1,0.0,4,4,2
4,5162,13891,14831,915,1035,80,4,9,5636,2020-01-01,...,SJC,WN,1,2,2020,1,0.0,4,4,1


In [123]:
new_df = pd.get_dummies(new_df,columns=['periodof_day'])
new_df = pd.get_dummies(new_df,columns=['dayof_theweek'])
new_df = pd.get_dummies(new_df,columns=['Nos flight per airport'])
new_df = pd.get_dummies(new_df,columns=['Nos_flight/carrier in airport'])
new_df = pd.get_dummies(new_df,columns=['origin_state_region'])
new_df = pd.get_dummies(new_df,columns=['WEEKEND'])
# new_df = pd.get_dummies(new_df,columns=['month'])

In [59]:
sc= preprocessing.Normalizer()
new_sam = sc.fit_transform(new_df._get_numeric_data())
df_sca=pd.DataFrame(new_sam, columns = new_df._get_numeric_data().columns)
df_sca.head()

,mkt_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,mkt_carrier_Cat,tail_num_Cat,month,year,...,Nos_flight/carrier in airport_1,Nos_flight/carrier in airport_2,Nos_flight/carrier in airport_3,Nos_flight/carrier in airport_4,origin_state_region_1,origin_state_region_2,origin_state_region_3,origin_state_region_4,WEEKEND_0.0,WEEKEND_1.0
0,0.266714,0.629233,0.669096,0.081989,0.088104,0.004303,0.000408,0.249727,0.000045,0.091502,...,0.0,0.0,0.0,0.000045,0.0,0.0,0.0,0.000045,0.000045,0.0
1,0.291760,0.645768,0.686678,0.053461,0.061364,0.004184,0.000418,0.104273,0.000046,0.093906,...,0.0,0.0,0.0,0.000046,0.0,0.0,0.0,0.000046,0.000046,0.0
2,0.213718,0.645662,0.689354,0.093891,0.099004,0.003254,0.000418,0.186527,0.000046,0.093891,...,0.0,0.0,0.0,0.000046,0.0,0.0,0.0,0.000046,0.000046,0.0
3,0.224347,0.654570,0.698865,0.063143,0.068562,0.003534,0.000424,0.122611,0.000047,0.095186,...,0.0,0.0,0.0,0.000047,0.0,0.0,0.0,0.000047,0.000047,0.0
4,0.236271,0.635809,0.678834,0.041881,0.047373,0.003662,0.000412,0.257967,0.000046,0.092458,...,0.0,0.0,0.0,0.000046,0.0,0.0,0.0,0.000046,0.000046,0.0


In [52]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn import metrics

In [124]:
new_df._get_numeric_data().shape

(659057, 36)

In [127]:
new_df.to_csv('X_predict.csv',index = False)

In [126]:
new_df.shape

(659057, 40)